In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
model_path = './model'
if not os.path.exists(model_path):
    os.mkdir(model_path)

In [ ]:
directory = "../input/home-depot-product-search-relevance"
pickle_directory = "../input/btl-home-depot-product-search-relevance/"

** Import libraries **

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import pickle
import keras as kr
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *
import xgboost
import gensim
from time import time
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess, tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Conv1D, Lambda, merge, Dense, Flatten,MaxPooling1D,Activation, Dropout,GlobalAveragePooling1D
from keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras import initializers,regularizers
import spacy
import re
import math
import gc
from keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from keras import backend as K
from sklearn.preprocessing import StandardScaler
from keras.utils.vis_utils import plot_model
import tensorflow as tf

In [ ]:
pip install plot_keras_history

In [ ]:
from plot_keras_history import plot_history

# Data

Read data

In [ ]:
train = pd.read_pickle(pickle_directory + "proccessed_train")
test = pd.read_pickle(pickle_directory + "proccessed_test")

In [ ]:
train = train.sort_values(['product_uid'])

Join tables

In [ ]:
text = ""
for i in range(0, len(train)):
    text += train.iloc[i]['product_title'] + ' ' + train.iloc[i]['search_term'] + ' ' + train.iloc[i]['product_description'] + ' '
for i in range(0, len(test)):
    text += test.iloc[i]['product_title'] + ' ' + test.iloc[i]['search_term'] + ' ' + test.iloc[i]['product_description'] + ' '
text_dataset = [text]

In [ ]:
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all['product_description'] = df_all['product_title'] + df_all['product_description'] 

Word embedding

In [ ]:
embeddings_index = dict()
f = open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
tokenizer = Tokenizer(num_words=66000)
tokenizer.fit_on_texts(text_dataset)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

cnt = 0
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        cnt = cnt + 1
print(vocab_size, ' ', cnt)

# MLP + extracted features

## Load features from previous notebook.

In [ ]:
infile = open("../input/btl-home-depot-product-search-relevance/features_18_4",'rb')
features = pickle.load(infile)

In [ ]:
df_extracted = features.drop(columns = ['product_title', 'product_description', 'search_term', 'material', 'brand'])
df_extracted.shape

In [ ]:
df_extracted.fillna(-1, inplace = True)

In [ ]:
train_X = df_extracted.drop(columns = ['relevance'])[:74067]
train_Y = df_extracted['relevance'][:74067]
test_X = df_extracted.drop(columns = ['relevance'])[74067:]

In [ ]:
x_scaler = StandardScaler().fit(train_X)
y_scaler = StandardScaler().fit(train_Y.values.reshape(-1,1))

train_X = x_scaler.transform(train_X)
test_X = x_scaler.transform(test_X)

In [ ]:
#Define metrics
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

## Model

In [ ]:
_input = Input(27,)

layer = Dense(32, activation="relu")(_input)
layer = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(layer)
layer = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(layer)
layer = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(layer)
layer = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(layer)
outputs = Dense(1)(layer)
 
dnn = Model(inputs=_input,outputs=outputs)
optimizer = optimizers.Adam(lr=0.0003)
dnn.compile(loss='mse',optimizer=optimizer,metrics = [RootMeanSquaredError()])
dnn.summary()

In [ ]:
patient = 40
filepath= "dnn.h5"
call_backs = [
    EarlyStopping(monitor='loss', patience=patient, mode='min', verbose=1),
    ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'),
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=patient/3, min_lr=0.000001, verbose=1, mode='min')
]

In [ ]:
%%time
history =  dnn.fit(train_X, train_Y, batch_size=64, epochs = 200, shuffle = True, verbose = 2, validation_split = 0.2, callbacks = call_backs)

In [ ]:
plot_history(history)

## Load best validate score weights and predict

In [ ]:
dnn.load_weights(filepath)
pred_dnn = dnn.predict(test_X)
print(pred_dnn.min(), ' ' , pred_dnn.max())

In [ ]:
pred = [i[0] for i in pred_dnn]

submission =pd.read_csv(directory + "/sample_submission.csv.zip")
pred = [min(3,i) for i in pred]
pred = [max(1, i) for i in pred]
submission['relevance'] = pred
submission.to_csv('submission_dnn.csv', index=False)
pred[:10]


# Semantic matching network

In [ ]:
MAX_TOKENS_NUM = 66054  # Maximum vocab size.
EMBEDDING_DIMS = 50
NUM_HIDDEN_NODE = 128
WORD_WINDOW_SIZE = 3
WORD_PER_DOC = 900
WORD_PER_QUERY = 15

### Element-wise product

In [ ]:
def hadamard_dot(x):
    x1 = x[0]
    x2 = x[1]
    out = x1 * x2
    return out

### Build parts of network

In [ ]:
def build_conv_embed_query(embed):
    conv = Conv1D(NUM_HIDDEN_NODE, 3, padding = 'same', activation="relu")(embed)
    conv = BatchNormalization()(conv)
    conv = MaxPooling1D(pool_size = WORD_PER_QUERY)(conv)
    conv = Dense(NUM_HIDDEN_NODE, activation = 'relu')(conv)
    layer = Lambda(lambda x: K.repeat_elements(x, 799, axis=1))(conv)
    return layer
def build_conv_embed_doc(embed):
    conv = Conv1D(NUM_HIDDEN_NODE, 3, activation="relu")(embed)
    conv = BatchNormalization()(conv)
    conv = MaxPooling1D(pool_size = 100,strides = 1)(conv)
    conv = Conv1D(NUM_HIDDEN_NODE, 1, activation="relu")(conv)
    return conv
def build_full_conected(layer):
    score = Conv1D(NUM_HIDDEN_NODE/8, 1, activation="relu")(layer)
    score = MaxPooling1D(pool_size = 3)(score)
    score = Flatten()(score)
    score = Dense(NUM_HIDDEN_NODE, activation="relu")(score)
    score = Dropout(0.5)(score)
    score = Dense(NUM_HIDDEN_NODE, activation="relu")(score)
    score = Dropout(0.5)(score)
    return score

## Model

In [ ]:
middle_input = Input(shape=(WORD_PER_QUERY,), name='search_term_input')
right_input = Input(shape=(WORD_PER_DOC,), name='desc_input')

query_embed2 = Embedding(MAX_TOKENS_NUM + 1, EMBEDDING_DIMS, input_length = WORD_PER_QUERY, weights=[embedding_matrix],trainable=True, name = 'search_term_embedding')
desc_embed = Embedding(MAX_TOKENS_NUM + 1, EMBEDDING_DIMS, input_length = WORD_PER_DOC, weights=[embedding_matrix],trainable=True, name = 'product_description_embedding')

encoded_query_2 = query_embed2(middle_input)
encoded_desc = desc_embed(right_input)

conv_query2 = build_conv_embed_query(encoded_query_2)
conv_desc = build_conv_embed_doc(encoded_desc)

h_dot_desc = Lambda(hadamard_dot)([conv_desc, conv_query2])

prediction = build_full_conected(h_dot_desc)
prediction = Dense(1, activation = 'tanh')(prediction)
prediction = Lambda(lambda x : x + 2)(prediction)

semantic_net = Model(inputs=[middle_input,right_input],outputs=prediction)

semantic_net.compile(loss='mse',optimizer=Adam(lr=0.0001),metrics = [RootMeanSquaredError()])
semantic_net.summary()

In [ ]:
plot_model(semantic_net, to_file='semantic_plot.png', show_shapes=True, show_layer_names=True)

## Prepare input and training

In [ ]:
search_input = df_all[:74067]['search_term']
title_input = df_all[:74067]['product_title']
descript_input = df_all[:74067]['product_description']

search_input = sequence.pad_sequences(tokenizer.texts_to_sequences(search_input), maxlen=WORD_PER_QUERY, padding='post', value = 0)
title_input = sequence.pad_sequences(tokenizer.texts_to_sequences(title_input), maxlen=45, padding='post', value = 1)
descript_input = sequence.pad_sequences(tokenizer.texts_to_sequences(descript_input), maxlen=WORD_PER_DOC, padding='post', value = 1)

targets = df_all[:74067]['relevance']

In [ ]:
patient = 20
filepath= "semantic_net"
call_backs = [
    EarlyStopping(monitor='val_loss', patience=patient, mode='min', verbose=1),
    ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only = True, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=patient/3, min_lr=0.000001, verbose=3, mode='min')
]

In [ ]:
%%time
history =  semantic_net.fit([search_input, descript_input], targets, batch_size=64, epochs = 200, validation_split = 0.3, shuffle = True, verbose =  2, callbacks=call_backs)

In [ ]:
plot_history(history)

## Load best validate score weights and make prediction

In [ ]:
search_test = df_all[74067:]['search_term']
title_test = df_all[74067:]['product_title']
descript_test = df_all[74067:]['product_description']

search_test = sequence.pad_sequences(tokenizer.texts_to_sequences(search_test), maxlen=WORD_PER_QUERY, padding='post')
title_test = sequence.pad_sequences(tokenizer.texts_to_sequences(title_test), maxlen=45, padding='post')
descript_test = sequence.pad_sequences(tokenizer.texts_to_sequences(descript_test), maxlen=WORD_PER_DOC, padding='post')

In [ ]:
semantic_net.load_weights(filepath)
preds = semantic_net.predict([search_test,descript_test])
print(preds.min(), ' ' , preds.max())

In [ ]:
pred = [i[0] for i in preds]

submission =pd.read_csv(directory + "/sample_submission.csv.zip")
pred = [min(3,i) for i in pred]
pred = [max(1, i) for i in pred]
submission['relevance'] = pred
submission.to_csv('submission_semantic.csv', index=False)
pred[:10]

# Interaction network

### Function to create table which (i,j) = 1 if query_term(i) == doc_term(j), 0 otherwise

In [ ]:
def xor_match(x):
    t1 = x[0]
    t2 = x[1]
    t1_shape = t1.get_shape()
    t2_shape = t2.get_shape()
    t1_expand = tf.stack([t1] * t2_shape[1], 2)
    t2_expand = tf.stack([t2] * t1_shape[1], 1)
    out_bool = tf.equal(t1_expand, t2_expand)
    out = tf.cast(out_bool, tf.float32)
    return out

## Model

In [ ]:
NUM_HIDDEN_NODE = 128
query = Input(shape=(WORD_PER_QUERY,), name='search_term_input')
doc = Input(shape=(WORD_PER_DOC,), name='desc_input')

xor = Lambda(xor_match)([query, doc])
conv = Conv1D(NUM_HIDDEN_NODE,1,activation='relu')(xor)
conv = MaxPooling1D(pool_size = 3, strides = 2)(conv)
layer = Flatten()(conv)
layer = Dense(NUM_HIDDEN_NODE, activation = 'relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(NUM_HIDDEN_NODE, activation = 'relu')(layer)
layer = Dropout(0.5)(layer)
prediction = Dense(1, activation = 'tanh')(layer)
prediction = Lambda(lambda x : x + 2)(prediction)

interaction_net = Model(inputs=[query,doc],outputs=prediction)

interaction_net.compile(loss='mse',optimizer=Adam(lr=0.00005),metrics = [RootMeanSquaredError()])
interaction_net.summary()

In [ ]:
plot_model(interaction_net, to_file='interaction_plot.png', show_shapes=True, show_layer_names=True)

## Training

In [ ]:
patient = 30
filepath= "interaction_net"
call_backs = [
    EarlyStopping(monitor='val_loss', patience=patient, mode='min', verbose=1),
    ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only = True, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=patient/4, min_lr=0.000001, verbose=3, mode='min')
]

In [ ]:
%%time
history =  interaction_net.fit([search_input, descript_input], targets, batch_size=64, epochs = 200, validation_split = 0.3, shuffle = True, verbose = 2, callbacks=call_backs)

In [ ]:
plot_history(history)

## Make prediction 

In [ ]:
interaction_net.load_weights(filepath)
preds = interaction_net.predict([search_test,descript_test])
print(preds.min(), ' ' , preds.max())

In [ ]:
pred = [i[0] for i in preds]

submission =pd.read_csv(directory + "/sample_submission.csv.zip")
pred = [min(3,i) for i in pred]
pred = [max(1, i) for i in pred]
submission['relevance'] = pred
submission.to_csv('submission_interaction.csv', index=False)
pred[:10]